In [15]:
import pandas as pd
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()



In [2]:
df = pd.read_pickle("../chat_log_data/pickle/twitch_chat_df.pkl")

In [13]:
df.head()

,index,game,channel,timestamp,user_name,chat_msg
0,55734,DOTA,admiralbulldog,2019-05-31 09:48:00,cy_dota2,Krappa
1,55735,DOTA,admiralbulldog,2019-05-31 09:48:00,asavell,ON STREAM :)
2,55758,DOTA,admiralbulldog,2019-05-31 09:48:00,gandor87,WHY IS IT PAUSED PepeHands
3,55759,DOTA,admiralbulldog,2019-05-31 09:48:00,nobody1564,LULW
4,55760,DOTA,admiralbulldog,2019-05-31 09:48:00,not_that_guy,@AdmiralBulldog what is the site where you can...


In [66]:
admiralbulldog = df[df['channel']=='admiralbulldog']
admiralbulldog.head()

,index,game,channel,timestamp,user_name,chat_msg
0,55734,DOTA,admiralbulldog,2019-05-31 09:48:00,cy_dota2,Krappa
1,55735,DOTA,admiralbulldog,2019-05-31 09:48:00,asavell,ON STREAM :)
2,55758,DOTA,admiralbulldog,2019-05-31 09:48:00,gandor87,WHY IS IT PAUSED PepeHands
3,55759,DOTA,admiralbulldog,2019-05-31 09:48:00,nobody1564,LULW
4,55760,DOTA,admiralbulldog,2019-05-31 09:48:00,not_that_guy,@AdmiralBulldog what is the site where you can...


In [32]:
stopwords_ = set(stopwords.words('english'))
punctuation_ = set(string.punctuation)

In [67]:
# sent_tokens = sent_tokenize(input_string)
admiralbulldog['sent_chat_msg'] = admiralbulldog['chat_msg'].apply(lambda x: sent_tokenize(x) )

# tokens = [sent for sent in map(word_tokenize, sent_tokens)]
admiralbulldog['word_chat_msg'] = admiralbulldog['chat_msg'].apply(lambda x: word_tokenize(x) )

# tokens_lower = [[word.lower() for word in sent]
#                  for sent in tokens]
# admiralbulldog['chat_msg'] = admiralbulldog['chat_msg'].apply(lambda x: )
admiralbulldog['word_chat_msg'] = admiralbulldog['chat_msg'].apply(lambda x: word_tokenize(x))

# def filter_tokens(sent):
#     return([w for w in sent if not w in stopwords_ and not w in punctuation_])

# tokens_filtered = list(map(filter_tokens, tokens_lower))


/Users/michaellin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/michaellin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [68]:
admiralbulldog['word_chat_msg']

0                                                 [Krappa]
1                                       [ON, STREAM, :, )]
2                         [WHY, IS, IT, PAUSED, PepeHands]
3                                                   [LULW]
4        [@, AdmiralBulldog, what, is, the, site, where...
5                                                   [D, :]
6                                                    [Pog]
7                                                 [Krappa]
8                                                   [WTFF]
9                                            [FeelsBadMan]
10                                [is, bulldog, jewish, ?]
11                                                   [Pog]
12                                          [T, D, :, XIC]
13                                   [I, BOUGHT, IT, :, )]
14                                                [Krappa]
15                                                  [LULW]
16                                                   [Po